In [1]:
import mne
import pandas as pd
import numpy as np
#import eelbrain
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from mne.decoding import SlidingEstimator, cross_val_multiscore
#import matplotlib.pyplot as plt
import glob
import os.path as op
from eelbrain._stats.stats import variability

subjectlist = ['A0421']
dates = {'A0421': '18Apr2023'}

tmin = -0.2  # in seconds
tmax = 0.6
trigger_delay = 30  # in ms/samples
shift_triggers = False
event_id = {'word_clean': 1,
            'word_noisy': 2,
            'word_symbols': 4,
            'letter_clean': 8,
            'letter_noisy': 16,
            'letter_symbol': 32}
# paths
data_root = '/Users/alr664/Desktop/'
mri_dir = data_root + 'brainquest/mri'
inv_dir = data_root +'/inv'

## Run the Linear Model on ecah subject to get coefficients for target predictors

In [ ]:
# testing can be 
testing = 'string-words'

#### This is the primary function for running the single trial linear regression on each subject's data. ####
# It takes as input:
# subject = the subject you are analyzing
# save_stcs = whether or not to write the stcs to file
# testing = the name of the predictor that you want to save. This can be 'noise',' string','string-words', or 'string-letters'

def run_source_LM(subject, date, save_stcs, testing):
    # read in the data
    raw = mne.io.read_raw_fif(data_root + 'meg/%s/%s_Tark_%s_NR-raw.fif' %(subject,subject,date))
    # find events
    events = mne.find_events(raw, min_duration = 0.002)
    print(len(events))
    events = [event for event in events if event[2] in event_id.values()]
    print(len(events))
    events = np.array(events)
    # shift triggers by 25 - need to check the number for each dataset
    if shift_triggers:
        # correct for trigger delay
        events[:, 0] += 25
    if make_plots:
        mne.viz.plot_events(events, event_id=event_id);
    # define the epochs
    epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax, event_id=event_id, preload=True, decim=5)
    # just keep meg channels
    epochs = epochs.pick_types(meg=True)
    # add metadata to epochs
    df = pd.DataFrame()
    df['trigger'] = epochs.events[:, 2]
    df['noise'] = [(ev in [2, 16])*1 for ev in epochs.events[:, 2]]
    df['symbol'] = [(ev in [4, 32])*1 for ev in epochs.events[:, 2]]
    df['fourchar'] = [(ev in [1, 2, 4])*1 for ev in epochs.events[:, 2]]
    epochs.metadata = df
    evokeds = []
    # make the evokeds, or average per each condition for inspection later
    for condition in epochs.event_id.keys():
       evokeds.append(epochs[condition].average())

    # define the forward and inverse solutions if they are not saved already
    if not op.exists(inv_dir + '%s_tark-meg-ico-4-inv.fif' %subject):
        covariance = mne.compute_covariance(epochs, tmin=-0.2,tmax = 0, method = 'empirical')
        trans = mne.read_trans(data_root + 'meg/%s/%s-trans.fif' %(subject,subject))
        src = mne.read_source_spaces(mri_dir + '/%s/bem/%s-ico-4-src.fif' %(subj,subj), subject = 'A0421')
        src.subject = 'A0421'
        bem = glob.glob(mri_dir + '/%s/bem/%s-inner_skull-bem-sol.fif' %(subject,subject))[0]
        if not op.exists(data_root + 'meg/%s/%s_decoding-fwd.fif' %(subject,subject)):
            fwd = mne.make_forward_solution(raw.info, trans, src, bem, meg=True, eeg=False, ignore_ref = True)
            fwd = mne.convert_forward_solution(fwd, force_fixed=True)
            mne.write_forward_solution(data_root + 'meg/%s/%s_decoding-fwd.fif' %(subject,subject),fwd,overwrite=True)
        fwd = mne.read_forward_solution(data_root + 'meg/%s/%s_decoding-fwd.fif' %(subject,subject))
        fwd = mne.convert_forward_solution(fwd, force_fixed=True)
        inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, covariance, depth=None, loose= 'auto', fixed=True)
        mne.minimum_norm.write_inverse_operator(inv_dir + '%s_tark-meg-ico-4-inv.fif' %subject, inv)
    else:
         inv = mne.minimum_norm.read_inverse_operator(inv_dir + '%s_tark-meg-ico-4-inv.fif' %subject)   
    # compute the STCS
    SNR = 2
    lambda2 = 1.0 / SNR ** 2
    stcs = mne.minimum_norm.apply_inverse_epochs(epochs, inv, lambda2, method='dSPM')
    # morph to fsaverage
    stcs[0].subject = 'A0421'
    morph = mne.compute_source_morph(stcs[0],subject_from=subject,subject_to='fsaverage',subjects_dir=mri_dir, spacing=4)
    morph.subject = 'A0421'
    fs_stcs = []
    for stc in stcs:
        stc.subject = 'A0421'
        fs_stcs.append(morph.apply(stc)) 
    # put all of it into a dataset
    ds = eelbrain.Dataset()
    ds['srcm'] = eelbrain.load.fiff.stc_ndvar(fs_stcs, subject= 'fsaverage', src='ico-4',subjects_dir = mri_dir)

    # save stcs if desired
    if save_stcs == True:
        dat = ds[np.array(epochs.metadata['trigger'] == 1)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_word_clean' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 2)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_word_noisy' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 4)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_word_symbol' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 8)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_letter_clean' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 16)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_letter_noisy' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 32)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_letter_symbol' %subject, dat)

    # the rest of these are loops to save the coefficients of interest, depending on what you're looking at    
    if testing == 'noise':
        # type two noise:
        ds_noise = ds[np.array(df['symbol'] == 0)]  # get all letter string stimuli
        df_noise = df[df['symbol'] == 0] 
        noise_type = df_noise['noise'].to_numpy()
        trialno = np.arange(1, len(ds_noise) + 1, 1)
        ds_noise['NoiseType'] = eelbrain.Factor(noise_type)
        ds_noise['TrialNo'] = eelbrain.Var(trialno)
        # run the test
        res = eelbrain.testnd.LM(ds_noise['srcm'],model='NoiseType + TrialNo', ds = ds_noise)
        coef = res.coefficient('NoiseType')
        np.save('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoNoise' %subject, coef.x)
    if testing == 'string':
        # type two string
        ds_string = ds[np.array(df['noise'] == 0)]
        df_string = df[df['noise'] == 0]
        string_type = df_string['symbol'].to_numpy()
        trialno = np.arange(1, len(ds_string) + 1, 1)
        ds_string['StringType'] = eelbrain.Factor(string_type)
        ds_string['TrialNo'] = eelbrain.Var(trialno)
        # run the test
        res = eelbrain.testnd.LM(ds_string['srcm'], model='StringType + TrialNo', ds= ds_string)
        coef = res.coefficient('StringType')
        np.save('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoString' %subject, coef.x)
    
    # sting
    if testing == 'string-words':
        # type two string: words only
        idx = [i in [1,4] for i in df['trigger'].values]                      
        idx = np.array(idx)
        ds_four = ds[idx]
        df_four = df[idx]
        string_type = df_four['symbol'].to_numpy()
        trialno = np.arange(1, len(ds_four) + 1, 1)
        ds_four['StringType'] = eelbrain.Factor(string_type)
        ds_four['TrialNo'] = eelbrain.Var(trialno)
        # run the test
        res = eelbrain.testnd.LM(ds_four['srcm'], model='StringType + TrialNo', ds= ds_four)
        coef = res.coefficient('StringType')
        np.save('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoString-Words' %subject, coef.x)
    elif testing == 'string-letters':
        # 1 unit only: letter = 8; symbol = 32; 
        idx = [i in [8,32] for i in df['trigger'].values]                      
        idx = np.array(idx)
        ds_one = ds[idx]
        df_one = df[idx]
        string_type = df_one['symbol'].to_numpy()
        trialno = np.arange(1, len(ds_one) + 1, 1)
        ds_one['StringType'] = eelbrain.Factor(string_type)
        ds_one['TrialNo'] = eelbrain.Var(trialno)
        # run the test
        res = eelbrain.testnd.LM(ds_one['srcm'], model='StringType + TrialNo', ds= ds_one)
        coef = res.coefficient('StringType')
        np.save('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoString-Letters' %subject, coef.x)        

In [ ]:
def run_source_LM(subject, date, save_stcs, testing):
    # read in the data
    raw = mne.io.read_raw_fif(data_root + 'meg/%s/%s_Tark_%s_NR-raw.fif' % (subject, subject, date))
    # find events
    events = mne.find_events(raw, min_duration=0.002)
    print(len(events))
    events = [event for event in events if event[2] in event_id.values()]
    print(len(events))
    events = np.array(events)
    # shift triggers by 25 - need to check the number for each dataset
    if shift_triggers:
        # correct for trigger delay
        events[:, 0] += 25
    if make_plots:
        mne.viz.plot_events(events, event_id=event_id)
    # define the epochs
    epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax, event_id=event_id, preload=True, decim=5)
    # just keep meg channels
    epochs = epochs.pick_types(meg=True)
    # add metadata to epochs
    df = pd.DataFrame()
    df['trigger'] = epochs.events[:, 2]
    df['noise'] = [(ev in [2, 16])*1 for ev in epochs.events[:, 2]]
    df['symbol'] = [(ev in [4, 32])*1 for ev in epochs.events[:, 2]]
    df['fourchar'] = [(ev in [1, 2, 4])*1 for ev in epochs.events[:, 2]]
    epochs.metadata = df
    evokeds = []
    # make the evokeds, or average per each condition for inspection later
    for condition in epochs.event_id.keys():
       evokeds.append(epochs[condition].average())

    # define the forward and inverse solutions if they are not saved already
    if not op.exists(inv_dir + '%s_tark-meg-ico-4-inv.fif' %subject):
        covariance = mne.compute_covariance(epochs, tmin=-0.2, tmax=0, method='empirical')
        trans = mne.read_trans(data_root + 'meg/%s/%s-trans.fif' % (subject, subject))
        src = mne.read_source_spaces(mri_dir + '/%s/bem/%s-ico-4-src.fif' % (subj, subj), subject='A0421')
        src.subject = 'A0421'
        bem = glob.glob(mri_dir + '/%s/bem/%s-inner_skull-bem-sol.fif' % (subject, subject))[0]
        if not op.exists(data_root + 'meg/%s/%s_decoding-fwd.fif' % (subject, subject)):
            fwd = mne.make_forward_solution(raw.info, trans, src, bem, meg=True, eeg=False, ignore_ref=True)
            fwd = mne.convert_forward_solution(fwd, force_fixed=True)
            mne.write_forward_solution(data_root + 'meg/%s/%s_decoding-fwd.fif' % (subject, subject), fwd, overwrite=True)
        fwd = mne.read_forward_solution(data_root + 'meg/%s/%s_decoding-fwd.fif' % (subject, subject))
        fwd = mne.convert_forward_solution(fwd, force_fixed=True)
        inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, covariance, depth=None, loose='auto', fixed=True)
        mne.minimum_norm.write_inverse_operator(inv_dir + '%s_tark-meg-ico-4-inv.fif' %subject, inv)
    else:
         inv = mne.minimum_norm.read_inverse_operator(inv_dir + '%s_tark-meg-ico-4-inv.fif' %subject)   
    # compute the STCS
    SNR = 2
    lambda2 = 1.0 / SNR ** 2
    stcs = mne.minimum_norm.apply_inverse_epochs(epochs, inv, lambda2, method='dSPM')
    # morph to fsaverage
    stcs[0].subject = 'A0421'
    morph = mne.compute_source_morph(stcs[0], subject_from=subject, subject_to='fsaverage', subjects_dir=mri_dir, spacing=4)
    morph.subject = 'A0421'
    fs_stcs = []
    for stc in stcs:
        stc.subject = 'A0421'
        fs_stcs.append(morph.apply(stc)) 
    # put all of it into a dataset
    ds = eelbrain.Dataset()
    ds['srcm'] = eelbrain.load.fiff.stc_ndvar(fs_stcs, subject='fsaverage', src='ico-4', subjects_dir=mri_dir)

    # save stcs if desired
    if save_stcs:
        dat = ds[np.array(epochs.metadata['trigger'] == 1)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_word_clean' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 2)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_word_noisy' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 4)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_word_symbol' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 8)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_letter_clean' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 16)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_letter_noisy' %subject, dat)
        dat = ds[np.array(epochs.metadata['trigger'] == 32)]['srcm'].mean('case').x
        np.save('/Users/alr664/Desktop/tark/avgs/%s_letter_symbol' %subject, dat)

    # the rest of these are loops to save the coefficients of interest, depending on what you're looking at    
    if testing == 'noise':
        # type two noise:
        ds_noise = ds[np.array(df['symbol'] == 0)]  # get all letter string stimuli
        df_noise = df[df['symbol'] == 0] 
        noise_type = df_noise['noise'].to_numpy()
        trialno = np.arange(1, 212, 1)
        print("Length of ds_noise:", len(ds_noise))
        print("Length of trialno (noise):", len(trialno))
        ds_noise['NoiseType'] = eelbrain.Factor(noise_type)
        ds_noise['TrialNo'] = eelbrain.Var(trialno)
        # run the test
        res = eelbrain.testnd.LM(ds_noise['srcm'], model='NoiseType + TrialNo', ds=ds_noise)
        coef = res.coefficient('NoiseType')
        np.save('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoNoise' %subject, coef.x)
    if testing == 'string':
        # type two string
        ds_string = ds[np.array(df['noise'] == 0)]
        df_string = df[df['noise'] == 0]
        string_type = df_string['symbol'].to_numpy()
        trialno = np.arange(1, 215, 1)
        print("Length of ds_string:", len(ds_string))
        print("Length of trialno (string):", len(trialno))
        ds_string['StringType'] = eelbrain.Factor(string_type)
        ds_string['TrialNo'] = eelbrain.Var(trialno)
        # run the test
        res = eelbrain.testnd.LM(ds_string['srcm'], model='StringType + TrialNo', ds=ds_string)
        coef = res.coefficient('StringType')
        np.save('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoString' %subject, coef.x)
    
    # string-words
    if testing == 'string-words':
        # type two string: words only
        idx = [i in [1, 4] for i in df['trigger'].values]                      
        idx = np.array(idx)
        ds_four = ds[idx]
        df_four = df[idx]
        string_type = df_four['symbol'].to_numpy()
        trialno = np.arange(1, 108, 1)
        print("Length of ds_four:", 108)
        print("Length of trialno (string-words):", len(trialno))
        ds_four['StringType'] = eelbrain.Factor(string_type)
        ds_four['TrialNo'] = eelbrain.Var(trialno)
        # run the test
        res = eelbrain.testnd.LM(ds_four['srcm'], model='StringType + TrialNo', ds=ds_four)
        coef = res.coefficient('StringType')
        np.save('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoString-Words' %subject, coef.x)
    elif testing == 'string-letters':
        # # 1 unit only: letter = 8; symbol = 32;
        idx = [i in [8, 32] for i in df['trigger'].values]
        idx = np.array(idx)
        ds_one = ds[idx]
        df_one = df[idx]
        string_type = df_one['symbol'].to_numpy()
        trialno = np.arange(1, 108, 1)
        print("Length of ds_one:", 108)
        print("Length of trialno (string-letters):", len(trialno))
        ds_one['StringType'] = eelbrain.Factor(string_type)
        ds_one['TrialNo'] = eelbrain.Var(trialno)
        # run the test
        res = eelbrain.testnd.LM(ds_one['srcm'], model='StringType + TrialNo', ds=ds_one)
        coef = res.coefficient('StringType')
        np.save('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoString-Letters' %subject, coef.x)

## Loop through the subjects to run it

In [ ]:
shift_triggers = True
make_plots = False
testing = 'string'
save_stcs = True
for subject in subjects:
    date = dates[subject]
    run_source_LM(subject, date, save_stcs, testing)

# Now examine the outputs for each variable

# Noise Type

In [ ]:
# load coefficients back in
src = mne.read_source_spaces(mri_dir + '/%s/bem/%s-ico-4-src.fif' %('fsaverage','fsaverage'))
stcs = []
for subject in subjectlist:
    coef = np.load('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoNoise.npy' %subject)
    stc = mne.SourceEstimate(data = coef, vertices = [src[0]['vertno'],src[1]['vertno']],subject='fsavearge',tmin=-0.2,tstep=0.005)
    stcs.append(stc)

Run the test on the coefficients from the whole sample of subjects:

In [ ]:
ds = eelbrain.Dataset()
ds['srcm'] = eelbrain.load.fiff.stc_ndvar(stcs, subject='fsaverage', src = 'ico-4',subjects_dir = mri_dir)
ds['Subject'] = eelbrain.Factor(subjects,random = True)
src = ds['srcm']
parc = mne.read_labels_from_annot('fsaverage','aparc',subjects_dir=mri_dir)
labels = [i for i in parc if i.name in ['lateraloccipital-lh','cuneus-lh','lingual-lh','pericalcarine-lh','fusiform-lh','middletemporal-lh','inferiortemporal-lh']]
label = labels[0] + labels[1] + labels[2] + labels[3] + labels[4] + labels[5] + labels[6]
label.name = 'ROI-lh'
src_region = src.sub(source=label)
ds['srcm'] = src_region
res = eelbrain.testnd.ttestonesample(ds['srcm'], popmean=0,match=None,ds =ds,tail = 0, samples = 10000, pmin=None,tfce=True, tstart=0.13, tstop=0.18)
print(res.clusters)

Typically you would inspect the clusters that prinnt out above, and then make labels from them for closer exaamination. like this:

In [ ]:
labels = eelbrain.labels_from_clusters(res.clusters[0:4,'cluster'])
mne.write_labels_to_annot(labels, parc='TypeTwoNoiseClusters',subject='fsaverage',subjects_dir = mri_dir,overwrite=True)
labels = mne.read_labels_from_annot(subject='fsaverage',parc='NoiseClusters',subjects_dir=mri_dir)

Then plot the time-course of the coefficients:

In [ ]:
labels = eelbrain.labels_from_clusters(res.clusters[:,'cluster'])
src_region = src.sub(source=labels[0])
ds['srcm'] = src_region
timecourse = ds['srcm'].mean('source')
import matplotlib.pyplot as plt
plt.plot(np.arange(-0.2,0.605,0.005),timecourse.mean('case').x)
plt.title('Beta Coefficient: First Cluster - Noise Type Two')

And alongside that, the time course of the mean activity by reading in the actual stcs (not the coefficients)

In [ ]:
src = mne.read_source_spaces(mri_dir + '/%s/bem/%s-ico-4-src.fif' %('fsaverage','fsaverage'))
conditionlist, subjectlist, stcs = [], [], []
conditions = ['letter_symbol','letter_noisy','letter_clean','word_symbol','word_noisy','word_clean']
for subject in subjects:
    for condition in conditions:
        dat = np.load('/Users/alr664/Desktop/tark/avgs/%s_%s.npy' %(subject, condition))
        stc = mne.SourceEstimate(data = dat,vertices = [src[0]['vertno'],src[1]['vertno']],subject='fsavearge',tmin=-0.2,tstep=0.005)
        stcs.append(stc)
        conditionlist.append(condition)
        subjectlist.append(subject)

In [ ]:
# This is all just extraction and plotting code

import matplotlib.pyplot as plt
ds = eelbrain.Dataset()
ds['srcm'] = eelbrain.load.fiff.stc_ndvar(stcs, subject='fsaverage', src = 'ico-4',subjects_dir = mri_dir)
ds['Subject'] = eelbrain.Factor(['A0421', 'A0421', 'A0421','A0421', 'A0421', 'A0421'], random=True)
#= eelbrain.Factor(subjectlist,random = True)
ds['Condition'] = eelbrain.Factor(conditionlist,random=True)
src = ds['srcm']
src_region = src.sub(source=labels[0])
ds['srcm'] = src_region
toplot = np.zeros((2, ds['srcm'].shape[2]))
uppers = np.zeros((2, ds['srcm'].shape[2]))
lowers = np.zeros((2, ds['srcm'].shape[2]))
#a = variability(ds['srcm'].mean('source'), x = ds['Condition'],match=ds['Subject'],pool=True,spec='1.5sem')
#conditions = [c for c in conditions if c in['letter_noisy','letter_clean','word_noisy','word_clean']]
sets = [['letter_noisy','word_noisy'],['letter_clean','word_clean']]
legendnames = ['High Noise','Low Noise']
fig = plt.figure(figsize=(18,12))
ax = fig.add_subplot(111)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_color('grey')
ax.spines['left'].set_color('grey')
plotlim = (-0.1,0.605,-0.8,0.605)
plt.rcParams.update({'font.size': 48})

ax.imshow([[0,0],[1,1]],cmap=plt.cm.Reds,interpolation='bicubic',extent = plotlim,alpha=0.05,origin='upper',aspect='auto')
colors = ['#900C3F','#0AA8D3']
for c in range(2):
    idx = ds['Condition'].isin(sets[c])
    toplot[c,:] = ds[idx]['srcm'].mean('source').mean('case').x
    #a = variability(y=ds[idx]['srcm'].mean('source').x,x=ds[idx]['Condition'],match=ds[idx]['Subject'],pool=True,spec='2sem')
    plt.plot(np.arange(-0.1,0.605,0.005),toplot[c,20:],label = legendnames[c],color=colors[c])
    #plt.fill_between(np.arange(-0.1,0.605,0.005),toplot[c,20:] + a[20:],toplot[c,20:] - a[20:],alpha=0.3,color=colors[c])

leg = plt.legend(loc='upper right')
for legobj in leg.legendHandles:
    legobj.set_linewidth(6.0)

# highlighing the cluster
plt.axhline(0.65,0.2/0.705,0.28/0.705,lw=6,color='orange') 
plt.text(0.117,0.65,'***',color='orange')

plt.xlabel('Time [s]', size=48)
plt.ylabel('Mean Activation [dSPM]', size=38)
#plt.savefig('/Users/grahamflick/Dropbox/Conferences_Presentations/2021/EPS_2021_WordsInContext_Symposium/images/Tark_TypeOneNoise.png')


# Examining the type two string type response

Now I'll do the same for the string type coefficients

In [ ]:
# load coefficients back in
src = mne.read_source_spaces(mri_dir + '/%s/bem/%s-ico-4-src.fif' %('fsaverage','fsaverage'))
stcs = []
for subject in subjectlist:
    coef = np.load('/Users/alr664/Desktop/tark/coefs/%s_TypeTwoString.npy' %subject)
    stc = mne.SourceEstimate(data = coef, vertices = [src[0]['vertno'],src[1]['vertno']],subject='fsavearge',tmin=-0.2,tstep=0.005)
    stcs.append(stc)

In [ ]:
split = True
ds = eelbrain.Dataset()
ds['srcm'] = eelbrain.load.fiff.stc_ndvar(stcs, subject='fsaverage', src = 'ico-4',subjects_dir = mri_dir)
ds['Subject'] = eelbrain.Factor(subjects,random = True)
src = ds['srcm']
parc = mne.read_labels_from_annot('fsaverage','aparc',subjects_dir=mri_dir)
labels = [i for i in parc if i.name in ['lateraloccipital-lh','cuneus-lh','lingual-lh','pericalcarine-lh','fusiform-lh','middletemporal-lh','inferiortemporal-lh']]
label = labels[0] + labels[1] + labels[2] + labels[3] + labels[4] + labels[5] + labels[6]
label.name = 'ROI-lh'
if split == True:
    split = np.min(label.pos[:,1]) + (1/2)*(np.ptp(label.pos[:,1]))
    index = label.pos[:,1] > split  
    label.vertices = label.vertices[index] 
    label.values = label.values[index] 
    label.pos = label.pos[index]  
src_region = src.sub(source=label)
ds['srcm'] = src_region
res = eelbrain.testnd.ttest_1samp(ds['srcm'], popmean=0,match=None,ds =ds,tail = 0, samples = 10000, pmin=None,tfce=True, tstart=0.18, tstop=0.3)
print(res.clusters)

In [ ]:
labels = eelbrain.labels_from_clusters(res.clusters[:,'cluster'])
mne.write_labels_to_annot(labels, parc='StringTypeClusters',subject='fsaverage',subjects_dir = mri_dir,overwrite=True)

In [ ]:
# plot the coefficient time course
src_region = src.sub(source=labels[0])
ds['srcm'] = src_region
timecourse = ds['srcm'].mean('source')
plt.plot(np.arange(-0.2,0.605,0.005),timecourse.mean('case').x)
plt.title('Beta Coefficient: First Cluster - String Type Two')

In [ ]:
# And now load the stcs back inn and plot the actual activity time course

#labels = mne.read_labels_from_annot(subject='fsaverage',subjects_dir=mri_dir,parc='StringTypeClusters')
import matplotlib.pyplot as plt
src = mne.read_source_spaces(mri_dir + '/%s/bem/%s-ico-4-src.fif' %('fsaverage','fsaverage'))
conditionlist, subjectlist, stcs = [], [], []
conditions = ['letter_symbol','letter_noisy','letter_clean','word_symbol','word_noisy','word_clean']
for subject in subjects:
    for condition in conditions:
        dat = np.load('/Users/alr664/Desktop/tark/avgs/%s_%s.npy' %(subject, condition))
        stc = mne.SourceEstimate(data = dat,vertices = [src[0]['vertno'],src[1]['vertno']],subject='fsaverage',tmin=-0.2,tstep=0.005)
        stcs.append(stc)
        conditionlist.append(condition)
        subjectlist.append(subject)
ds = eelbrain.Dataset()
ds['srcm'] = eelbrain.load.fiff.stc_ndvar(stcs, subject='fsaverage', src = 'ico-4',subjects_dir = mri_dir)
ds['Subject'] = eelbrain.Factor(subjectlist,random = True)
ds['Condition'] = eelbrain.Factor(conditionlist,random=True)
src = ds['srcm']
src_region = src.sub(source=labels[0])

ds['srcm'] = src_region
fig = plt.figure(figsize=(18,12))
ax = fig.add_subplot(111)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_color('grey')
ax.spines['left'].set_color('grey')
plt.rcParams.update({'font.size': 48})


toplot = np.zeros((2, ds['srcm'].shape[2]))
idx = ds['Condition'].isin(['letter_clean','word_clean','letter_symbol','word_symbol'])
a = variability(ds[idx]['srcm'].mean('source'), x = ds[idx]['Condition'],match=ds[idx]['Subject'],pool=True,spec='1.5sem')
sets = [['letter_clean','word_clean'],['letter_symbol','word_symbol']]
legendnames = ['Letters','Symbols']
colors = ['#0AA8D3','#900C3F']
plotlim = (-0.1,0.605,-0.25,0.2)
ax.imshow([[0,0],[1,1]],cmap=plt.cm.Reds,interpolation='bicubic',extent = plotlim,alpha=0.05,origin='upper',aspect='auto')

for c in range(2):
    idx = ds['Condition'].isin(sets[c])
    toplot[c,:] = ds[idx]['srcm'].mean('source').mean('case').x
    plt.plot(np.arange(-0.1,0.605,0.005),toplot[c,20:],label=legendnames[c],color=colors[c])
    #plt.fill_between(np.arange(-0.1,0.605,0.005),toplot[c,20:] + a[20:], toplot[c,20:] - a[20:],alpha=0.3,color=colors[c])
plt.axhline(0.18,0.31/0.705,0.4/0.705,lw=6,color='orange') 
plt.text(0.23,0.185,'***',color='orange')
plt.xlabel('Time [s]',size=48)
plt.ylabel('Mean Activation [dSPM]',size=38)
leg = plt.legend(loc='lower left')
for legobj in leg.legendHandles:
    legobj.set_linewidth(6.0)
#plt.title('Letters and Words')
plt.savefig('/Users/grahamflick/Dropbox/Conferences_Presentations/2021/EPS_2021_WordsInContext_Symposium/images/Tark_StringType_Both.png')